In [12]:
import os
import numpy as np
import pandas as pd
#import torch

from collections import Counter
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
# all.review.vec.txt
with open("./data/embed_vec.txt", 'r', encoding="UTF8") as file:
    a = file.readlines()

In [14]:
# index_word : about all.review.vec.txt, word
# emb_vec : about all.review.vec.txt, embedding vector
index_word = []
emb_vec = np.zeros((56050, 100))
for i in range(56050):
    index_word.append(a[i].split()[0])
    for j in range(100):
        emb_vec[i, j] = float(a[i].split()[j+1])

In [15]:
# word_list : about reviews, every words
word_list = []
tokens = []
sentence_max = []
for i in range(2000):
    f = open("./data/train/all/{}".format(i), 'r', encoding="UTF8")
    reviews = f.readline()
    token = word_tokenize(reviews)
    tokens += token
    sentence_max.append(len(token))
max_len = np.max(sentence_max)
print(max_len)
for i in reversed(range(len(tokens))):
    if (tokens[i] not in index_word):
        del tokens[i]

3817


In [18]:
# tag_count : about reviews, frequncy of 10000 words
# tags : about reviews, frequnt 10000 words
count = Counter(tokens)
tag_count = []
tags = []
for n, c in count.most_common(10000):
    dics = {'tag': n, 'count': c}
    tag_count.append(dics)
    tags.append(dics['tag'])

In [19]:
# embedding : about all.review.vec.txt (word, embedding vector)
embedding = {}
for i in range(len(index_word)):
    embedding[index_word[i]] = emb_vec[i, :]

In [20]:
final = pd.DataFrame({"index": [], "embedding_vector": []})

for i in range(10000):
    final.loc[i] = [i, embedding[tags[i]].tolist()]


C:\Users\user\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


labeling

In [22]:
neg_dir = './data/train/negative/'
pos_dir = './data/train/positive/'

neg_list = os.listdir(neg_dir)
pos_list = os.listdir(pos_dir)

neg_lists = [int(str) for str in neg_list]
pos_lists = [int(str) for str in pos_list]

neg_lists = sorted(neg_lists)
pos_lists = sorted(pos_lists)

In [39]:
tags_word2idx = {}
for i in range(len(tags)):
    tags_word2idx[tags[i]] = i

train

In [62]:
train = np.full((2000, 3774), 10001) # 3817

In [63]:
tags_word2idx = {}
for i in range(len(tags)):
    tags_word2idx[tags[i]] = i

In [70]:
for i in range(2000):
    f = open("./data/train/all/{}".format(i), 'r', encoding="UTF8")
    reviews = f.readline()
    if i in neg_lists:
        label = 0
    else:
        label = 1
    token = word_tokenize(reviews)
    for k in reversed(range(len(token))):
        if (token[k] not in index_word):
            del token[k]
        
    for j in range(len(token)):
        if token[j] in tags:
            train[i, j] = tags_word2idx[token[j]]
        else:
            train[i, j] = 10000
        train[i, 3773] = label

In [73]:
np.save("./train", train)

test

In [76]:
test = np.full((2000, 3774), 10001)

tags_word2idx = {}
for i in range(len(tags)):
    tags_word2idx[tags[i]] = i

In [77]:
for i in range(2000):
    f = open("./data/test/{}".format(i), 'r', encoding="UTF8")
    reviews = f.readline()
    if i in neg_lists:
        label = 0
    else:
        label = 1
    token = word_tokenize(reviews)
    for k in reversed(range(len(token))):
        if (token[k] not in index_word):
            del token[k]
            
    for j in range(len(token)):
        if token[j] in tags:
            test[i, j] = tags_word2idx[token[j]]
        else:
            test[i, j] = 10000
        test[i, 3773] = label

In [54]:
np.save("./test", test)